In [1]:
#default_exp merge
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Merge
> Utility functions to correcly merge and clean the various data sources

In [3]:
#export
from decimal import Decimal
import pandas as pd
from geopy.distance import geodesic

from anateldb.constants import ENTIDADES
from anateldb.format import input_coordenates, scrape_dataframe, df_optimize

In [4]:
#export 
def _merge_dfs(df1, df2, on, how="left"):
    """Merge two dataframes with the same columns and records"""
    df = pd.merge(df1, df2, on=on, how=how)
    x = df.Description_x.notna()
    y = df.Description_y.notna()
    df.loc[x & y, "Description"] = (
        df.loc[x & y, "Description_x"] + " | " + df.loc[x & y, "Description_y"]
    )
    df.loc[x & ~y, "Description"] = df.loc[x & ~y, "Description_x"]
    df.loc[~x & y, "Description"] = df.loc[~x & y, "Description_y"]
    df.loc[x & y, "Latitude"] = (
        df.loc[x & y, "Latitude_x"] + df.loc[x & y, "Latitude_y"]
    ) / 2
    df.loc[x & y, "Longitude"] = (
        df.loc[x & y, "Longitude_x"] + df.loc[x & y, "Longitude_y"]
    ) / 2
    df.loc[x & ~y, "Latitude"] = df.loc[x & ~y, "Latitude_x"]
    df.loc[x & ~y, "Longitude"] = df.loc[x & ~y, "Longitude_x"]
    df.loc[~x & y, "Latitude"] = df.loc[~x & y, "Latitude_y"]
    df.loc[~x & y, "Longitude"] = df.loc[~x & y, "Longitude_y"]
    if "Service_x" in df.columns and "Service_y" in df.columns:
        df.loc[x, "Service"] = df.loc[x, "Service_x"]
        df.loc[~x & y, "Service"] = df.loc[~x & y, "Service_y"]
    return df.loc[:, [c for c in df.columns if "_" not in c]]


def aero_common(dfa, dfb, dfc):
    cols = ["Frequency", "Station"]
    a = dfa[dfa.Station != -1].reset_index(drop=True)
    b = dfb[dfb.Station != -1].reset_index(drop=True)
    c = dfc[dfc.Station != -1].reset_index(drop=True)
    common = _merge_dfs(a, b, cols, how="outer")
    common = _merge_dfs(common, c, cols, how="outer")
    common.drop_duplicates(inplace=True, keep="first")
    return common.reset_index(drop=True)

In [5]:
#export
def aero_new(dfa, dfb, dfc, dist=500):
    cols = ["Frequency"]
    a = (
        dfa[dfa.Station == -1]
        .drop(columns=["Service", "Station"])
        .reset_index(drop=True)
    )
    b = (
        dfb[dfb.Station == -1]
        .drop(columns=["Service", "Station"])
        .reset_index(drop=True)
    )
    c = (
        dfc[dfc.Station == -1]
        .drop(columns=["Service", "Station"])
        .reset_index(drop=True)
    )
    abc = (
        pd.merge(a, b, on=cols, how="outer")
        .merge(c, on=cols, how="outer")
        .reset_index(drop=True)
        .drop_duplicates(keep="first")
    )
    x = abc.Description_x.notna()
    y = abc.Description_y.notna()
    z = abc.Description.notna()
    new = pd.DataFrame(columns=["Frequency", "Latitude", "Longitude", "Description"])

    for c, n in zip([(x & ~y & ~z), (~x & y & ~z), (~x & ~y & z)], ["_x", "_y", ""]):
        temp = abc.loc[
            c, ["Frequency", f"Latitude{n}", f"Longitude{n}", f"Description{n}"]
        ]
        temp.columns = ["Frequency", "Latitude", "Longitude", "Description"]
        new = pd.concat([new, temp], ignore_index=True)

    new = new.drop_duplicates(keep="first").reset_index(drop=True)

    for c, (left, right) in zip(
        [(x & y & ~z), (x & ~y & z), (~x & y & z)],
        [("_x", "_y"), ("_x", ""), ("_y", "")],
    ):
        for row in abc[c].itertuples():
            d = geodesic(
                (getattr(row, f"Latitude{left}"), getattr(row, f"Longitude{left}")),
                (getattr(row, f"Latitude{right}"), getattr(row, f"Longitude{right}")),
            ).m
            if d < dist:
                f = row.Frequency
                lat = (
                    getattr(row, f"Latitude{left}") + getattr(row, f"Latitude{right}")
                ) / 2
                lon = (
                    getattr(row, f"Longitude{left}") + getattr(row, f"Longitude{right}")
                ) / 2
                d = (
                    getattr(row, f"Description{left}")
                    + " | "
                    + getattr(row, f"Description{right}")
                )
                new = pd.concat(
                    [
                        new,
                        pd.DataFrame(
                            [[f, lat, lon, d]],
                            columns=[
                                "Frequency",
                                "Latitude",
                                "Longitude",
                                "Description",
                            ],
                        ),
                    ],
                    ignore_index=True,
                )
            else:
                l = (
                    abc.loc[
                        row.Index,
                        [
                            "Frequency",
                            f"Latitude{left}",
                            f"Longitude{left}",
                            f"Description{left}",
                        ],
                    ]
                    .to_frame()
                    .T
                )
                l.columns = new.columns
                r = (
                    abc.loc[
                        row.Index,
                        [
                            "Frequency",
                            f"Latitude{right}",
                            f"Longitude{right}",
                            f"Description{right}",
                        ],
                    ]
                    .to_frame()
                    .T
                )
                r.columns = new.columns
                new = pd.concat([new, l, r], ignore_index=True)

    new = new.drop_duplicates(keep="first").reset_index(drop=True)

    for row in abc[x & y & z].itertuples():
        d1 = geodesic(
            (getattr(row, "Latitude_x"), getattr(row, "Longitude_x")),
            (getattr(row, "Latitude_y"), getattr(row, "Longitude_y")),
        ).m
        d2 = geodesic(
            (getattr(row, "Latitude_x"), getattr(row, "Longitude_x")),
            (getattr(row, "Latitude"), getattr(row, "Longitude")),
        ).m
        d3 = geodesic(
            (getattr(row, "Latitude_y"), getattr(row, "Longitude_y")),
            (getattr(row, "Latitude"), getattr(row, "Longitude")),
        ).m
        if d1 < dist and d2 < dist and d3 < dist:
            f = row.Frequency
            lat = (row.Latitude_x + row.Latitude_y + row.Latitude) / 3
            lon = (row.Longitude_x + row.Longitude_y + row.Longitude) / 3
            d = " | ".join([row.Description_x, row.Description_y, row.Description])
            new = pd.concat(
                [
                    new,
                    pd.DataFrame(
                        [[f, lat, lon, d]],
                        columns=["Frequency", "Latitude", "Longitude", "Description"],
                    ),
                ],
                ignore_index=True,
            )
        elif d1 < dist and d2 < dist:
            f = row.Frequency
            lat = (row.Latitude_x + row.Latitude_y) / 2
            lon = (row.Longitude_x + row.Longitude_y) / 2
            d = " | ".join([row.Description_x, row.Description_y])
            new = pd.concat(
                [
                    new,
                    pd.DataFrame(
                        [[f, lat, lon, d]],
                        columns=["Frequency", "Latitude", "Longitude", "Description"],
                    ),
                ],
                ignore_index=True,
            )
        elif d1 < dist and d3 < dist:
            f = row.Frequency
            lat = (row.Latitude_x + row.Latitude) / 2
            lon = (row.Longitude_x + row.Longitude) / 2
            d = " | ".join([row.Description_x, row.Description])
            new = pd.concat(
                [
                    new,
                    pd.DataFrame(
                        [[f, lat, lon, d]],
                        columns=["Frequency", "Latitude", "Longitude", "Description"],
                    ),
                ],
                ignore_index=True,
            )
        elif d2 < dist and d3 < dist:
            f = row.Frequency
            lat = (row.Latitude_y + row.Latitude) / 2
            lon = (row.Longitude_y + row.Longitude) / 2
            d = " | ".join([row.Description_y, row.Description])
            new = pd.concat(
                [
                    new,
                    pd.DataFrame(
                        [[f, lat, lon, d]],
                        columns=["Frequency", "Latitude", "Longitude", "Description"],
                    ),
                ],
                ignore_index=True,
            )
        else:
            l = (
                abc.loc[
                    row.Index,
                    ["Frequency", "Latitude_x", "Longitude_x", "Description_x"],
                ]
                .to_frame()
                .T
            )
            l.columns = new.columns
            r = (
                abc.loc[
                    row.Index,
                    ["Frequency", "Latitude_y", "Longitude_y", "Description_y"],
                ]
                .to_frame()
                .T
            )
            r.columns = new.columns
            s = (
                abc.loc[
                    row.Index, ["Frequency", "Latitude", "Longitude", "Description"]
                ]
                .to_frame()
                .T
            )
            s.columns = new.columns
            new = pd.concat([new, l, r, s], ignore_index=True)

    new = new.drop_duplicates(keep="first").reset_index(drop=True)
    new["Service"] = pd.NA
    new["Station"] = pd.NA
    return new

In [6]:
#export
def merge_aero(df, common, new):
    """Mescla a base da Anatel com as tabelas retiradas da Aeronáutica"""
    common = common.loc[:, ["Frequency", "Description", "Service", "Station"]]
    df["Description"] = df.Description.astype("string")
    df = pd.merge(df, common, on=["Frequency", "Service", "Station"], how="left")
    df.loc[df.Description_y.notna(), "Description_x"] = (
        df.loc[df.Description_y.notna(), "Description_x"]
        + " | "
        + df.loc[df.Description_y.notna(), "Description_y"]
    )
    df.drop(columns=["Description_y"], inplace=True)
    df.rename(columns={"Description_x": "Description"}, inplace=True)
    new['Class'] = pd.NA
    new['BW'] = pd.NA
    return (
        pd.concat([df, new], ignore_index=True)
        .sort_values("Frequency")
        .reset_index(drop=True)
    )

In [7]:
#export
def clean_mosaico(df, pasta):
    """Clean the merged dataframe with the data from the MOSAICO page"""
    COLS = [c for c in df.columns if "_x" in c]
    for col in COLS:
        col_x = col
        col_y = col.split("_")[0] + "_y"
        df.loc[df[col_x].isna(), col_x] = df.loc[df[col_x].isna(), col_y]
        df.loc[df[col_y].isna(), col_y] = df.loc[df[col_y].isna(), col_x]
        if df[col_x].notna().sum() > df[col_y].notna().sum():
            a, b = col_x, col_y
        else:
            a, b = col_y, col_x
        df.drop(b, axis=1, inplace=True)
        df.rename({a: a[:-2]}, axis=1, inplace=True)

    df.loc[df.Latitude_Transmissor == "", "Latitude_Transmissor"] = df.loc[
        df.Latitude_Transmissor == "", "Latitude_Estação"
    ]
    df.loc[df.Longitude_Transmissor == "", "Longitude_Transmissor"] = df.loc[
        df.Longitude_Transmissor == "", "Longitude_Estação"
    ]
    df.loc[df.Latitude_Transmissor.isna(), "Latitude_Transmissor"] = df.loc[
        df.Latitude_Transmissor.isna(), "Latitude_Estação"
    ]
    df.loc[df.Longitude_Transmissor.isna(), "Longitude_Transmissor"] = df.loc[
        df.Longitude_Transmissor.isna(), "Longitude_Estação"
    ]
    df.drop(["Latitude_Estação", "Longitude_Estação"], axis=1, inplace=True)
    df.rename(
        columns={
            "Latitude_Transmissor": "Latitude",
            "Longitude_Transmissor": "Longitude",
        },
        inplace=True,
    )

    df = input_coordenates(df, pasta)
    df.loc[:, "Frequência"] = df.Frequência.str.replace(",", ".")
    if freq_nans := df[df.Frequência.isna()].Id.tolist():
        complement_df = scrape_dataframe(freq_nans)
        df.loc[
            df.Frequência.isna(),
            [
                "Status",
                "Entidade",
                "Fistel",
                "Frequência",
                "Classe",
                "Num_Serviço",
                "Município",
                "UF",
            ],
        ] = complement_df.values

    df.loc[:, "Frequência"] = df.Frequência.astype("float")
    df.loc[df.Serviço == "OM", "Frequência"] = df.loc[
        df.Serviço == "OM", "Frequência"
    ].apply(lambda x: Decimal(x) / Decimal(1000))
    df.loc[:, "Validade_RF"] = df.Validade_RF.astype("string").str.slice(0, 10)
    return df.drop_duplicates(keep="first").reset_index(drop=True)

In [8]:
from nbdev.export import notebook2script; notebook2script()

Converted constants.ipynb.
Converted filter.ipynb.
Converted format.ipynb.
Converted index.ipynb.
Converted merge.ipynb.
Converted query.ipynb.
Converted read.ipynb.
